In [1]:
import json
import os
import requests
import pandas as pd
import numpy as np
import time
from dotenv import load_dotenv

In [2]:
def cargar_configuracion():
    """
    Carga todas las configuraciones necesarias desde la carpeta /references.
    """
    print("--- 1. Cargando configuración ---")
    load_dotenv()
    token = os.getenv("INEGI_TOKEN")
    if not token:
        raise ValueError("No se encontró el token de INEGI...")

    ruta_referencias = '../references/'
    
    try:
        ruta_municipales = os.path.join(ruta_referencias, 'diccionario_inegi_municipio.json')
        ruta_contexto = os.path.join(ruta_referencias, 'diccionario_inegi_contexto.json') 
        ruta_dict_municipios = os.path.join(ruta_referencias, 'diccionario_municipios_sonora.json')

        with open(ruta_municipales, 'r', encoding='utf-8') as f:
            config_municipales = json.load(f)
        with open(ruta_contexto, 'r', encoding='utf-8') as f:
            config_contexto = json.load(f) 
        with open(ruta_dict_municipios, 'r', encoding='utf-8') as f:
            municipios = json.load(f)
            
    except FileNotFoundError as e:
        raise Exception(f"Error: No se encontró un archivo de configuración. Detalle: {e}")
        
    print("✅ Configuración cargada exitosamente.")
    return token, config_municipales, config_contexto, municipios

In [3]:
def descargar_datos_municipales(token, config_municipales, municipios):
    """
    Descarga y procesa la serie histórica completa para todos 
    los indicadores a nivel municipal.
    """
    print("\n--- 2. Descargando datos municipales (serie histórica completa) ---")
    CLAVE_SONORA = '07000026'
    ruta_external = '../data/external/'
    os.makedirs(ruta_external, exist_ok=True)

    for indicador in config_municipales['indicadores_municipales']:
        nombre_indicador = indicador['nombre']
        id_indicador = indicador['id_inegi']

        ruta_csv = os.path.join(ruta_external, f"{nombre_indicador}.csv")

        # Verificamos si el archivo final ya existe
        if os.path.exists(ruta_csv):
            print(f"\nEl archivo '{nombre_indicador}.csv' ya existe. Se omite.")
            continue 
        
        # Si no existe, el resto del código se ejecuta normalmente
        print(f"\nProcesando indicador: {nombre_indicador}...")
        
        datos_de_este_indicador = []
        for nombre_mun, codigo_mun in municipios.items():
            ubicacion = CLAVE_SONORA + codigo_mun
            url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{id_indicador}/es/{ubicacion}/false/BISE/2.0/{token}?type=json"
            
            try:
                response = requests.get(url)
                response.raise_for_status()
                data = response.json()
                observaciones = data['Series'][0]['OBSERVATIONS']
                
                # Verificamos si la lista de observaciones no está vacía
                if observaciones:
                    # Recorremos CADA observación (cada año) en la lista
                    for obs in observaciones:
                        valor = obs['OBS_VALUE']
                        periodo = obs['TIME_PERIOD']
                        
                        # Guardamos el valor si no es nulo, o NaN si es nulo
                        if valor is not None:
                            datos_de_este_indicador.append({'municipio': nombre_mun, 'periodo': periodo, 'valor': float(valor)})
                        else:
                            datos_de_este_indicador.append({'municipio': nombre_mun, 'periodo': periodo, 'valor': np.nan})
                else:
                    print(f" -> Advertencia: No se encontraron observaciones para {nombre_mun}.")

            except Exception as e:
                print(f" -> Error al consultar {nombre_mun}: {e}")
            
            time.sleep(0.1)

        if datos_de_este_indicador:
            df = pd.DataFrame(datos_de_este_indicador)
            df = df[['municipio', 'periodo', 'valor']] # Reordenamos
            ruta_csv = os.path.join(ruta_external, f"{nombre_indicador}.csv")
            df.to_csv(ruta_csv, index=False, encoding='utf-8')
            print(f" -> ✅ Archivo '{nombre_indicador}.csv' guardado con {len(df)} registros.")

In [4]:
def descargar_datos_contexto(token, config_contexto):
    """
    Descarga y procesa todos los indicadores de contexto (estatales y nacionales).
    """
    print("\n--- 3. Descargando datos de contexto (Estatales/Nacionales) ---")
    CLAVE_GEO_SONORA = '07000026'
    CLAVE_GEO_NACIONAL = '0700'
    ruta_external = '../data/external/'
    os.makedirs(ruta_external, exist_ok=True)

    for indicador in config_contexto['indicadores_contexto']:
        nombre_indicador = indicador['nombre']
        id_indicador = indicador['id_inegi']
        
        ruta_csv = os.path.join(ruta_external, f"{nombre_indicador}.csv")
        if os.path.exists(ruta_csv):
            print(f"\nEl archivo '{nombre_indicador}.csv' ya existe. Se omite.")
            continue

        nivel_geo = indicador.get('nivel_geografico', 'estatal')
        fuente_api = indicador.get('fuente_api', 'BISE')
        
        if nivel_geo == 'nacional':
            ubicacion = CLAVE_GEO_NACIONAL
        else:
            ubicacion = CLAVE_GEO_SONORA
        
        print(f"\nProcesando indicador '{nombre_indicador}' desde {fuente_api}...")
        
        url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{id_indicador}/es/{ubicacion}/false/{fuente_api}/2.0/{token}?type=json"
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            observaciones = data['Series'][0]['OBSERVATIONS']
            datos_limpios = [{'periodo': obs['TIME_PERIOD'], 'valor': float(obs['OBS_VALUE'])} for obs in observaciones if obs['OBS_VALUE'] is not None]
            
            df = pd.DataFrame(datos_limpios)
            ruta_csv = os.path.join(ruta_external, f"{nombre_indicador}.csv")
            df.to_csv(ruta_csv, index=False, encoding='utf-8')
            print(f" -> ✅ Archivo '{nombre_indicador}.csv' guardado.")
        except Exception as e:
            print(f" -> Error al procesar el indicador {nombre_indicador}: {e}")

In [6]:
try:
    api_token, conf_municipales, conf_contexto, dict_municipios = cargar_configuracion()
    descargar_datos_municipales(api_token, conf_municipales, dict_municipios)
    descargar_datos_contexto(api_token, conf_contexto)
    print("\n--- 🎉 ¡Proceso finalizado! ---")
except Exception as e:
    print(f"\n--- ❌ Ocurrió un error en el proceso principal: {e} ---")

--- 1. Cargando configuración ---
✅ Configuración cargada exitosamente.

--- 2. Descargando datos municipales (serie histórica completa) ---

Procesando indicador: grado_promedio_escolaridad...
 -> ✅ Archivo 'grado_promedio_escolaridad.csv' guardado con 436 registros.

Procesando indicador: porcentaje_alfabetas...
 -> ✅ Archivo 'porcentaje_alfabetas.csv' guardado con 292 registros.

Procesando indicador: poblacion_asiste_escuela...
 -> ✅ Archivo 'poblacion_asiste_escuela.csv' guardado con 363 registros.

Procesando indicador: total_viviendas_habitadas...
 -> ✅ Archivo 'total_viviendas_habitadas.csv' guardado con 437 registros.

Procesando indicador: viviendas_con_internet...
 -> ✅ Archivo 'viviendas_con_internet.csv' guardado con 146 registros.

Procesando indicador: viviendas_con_automovil...
 -> ✅ Archivo 'viviendas_con_automovil.csv' guardado con 73 registros.

Procesando indicador: poblacion_total...
 -> ✅ Archivo 'poblacion_total.csv' guardado con 373 registros.

--- 3. Descargand